In [1]:
import sys
import os
import numpy as np
import operator

## Read Data

In [2]:
# reading training data
en_train = open('./training/hansards.36.2.e').read().splitlines()
fr_train = open('./training/hansards.36.2.f').read().splitlines()

# reading validation data
en_val = open('./validation/dev.e').read().splitlines()
fr_val = open('./validation/dev.f').read().splitlines()

print(len(en_train), len(fr_train), len(en_val), len(fr_val))

231164 231164 37 37


In [3]:
en_train[0]

'36 th Parliament , 2 nd Session '

In [4]:
fr_train[0]

'36 e Législature , 2 ième Session '

In [5]:
theta_0 = np.load('theta_0.npy').item()
theta_10 = np.load('theta_10.npy').item()
count_f_e_0 = np.load('count_f_e_0.npy').item()
count_e_0 = np.load('count_e_0.npy').item()

'''
np.save('theta_0.npy', theta_0) 
np.save('count_f_e_0.npy', count_f_e) 
np.save('count_e_0.npy', count_e)
np.save('theta_10.npy', theta) 
'''

## Preprocessing

In [6]:
def preprocess(s):
    a = ''.join(x for x in s if x.isalpha() or x == " ").lower()
    b = " ".join(a.split())
    return b.split(" ")

In [7]:
s = en_train[149]
s

'the Government will : '

In [8]:
preprocess(s)

['the', 'government', 'will']

## Initialize theta 0

In [9]:
def init_theta(en_train, fr_train):
    theta_0 = {}
    count_f_e = {}
    count_e = {}
    
    N= len(en_train)
    for n in range(N):
        
        if (n % 1000) == 0:
            print(n)
            
        en_n = en_train[n]
        cleaned_en_n = preprocess(en_n)
        fr_n = fr_train[n]
        cleaned_fr_n = preprocess(fr_n)
        
        for j_en_word in cleaned_en_n:
            if j_en_word not in theta_0:
                theta_0[j_en_word] = {}
            if j_en_word not in count_e:
                count_e[j_en_word] = 0
            for i_fr_word in cleaned_fr_n:
                if i_fr_word not in theta_0[j_en_word]:
                    theta_0[j_en_word][i_fr_word] = 0
                    
                pair_f_e = []
                pair_f_e.append(i_fr_word)
                pair_f_e.append(j_en_word)
                pair_f_e = tuple(pair_f_e)
                if pair_f_e not in count_f_e:
                    count_f_e[pair_f_e] = 0
                    
    for en_word in theta_0.keys():
        count = len(theta_0[en_word].keys())
        for fr_word in theta_0[en_word].keys():
            theta_0[en_word][fr_word] = 1/count
            
    return theta_0, count_f_e, count_e

In [11]:
theta_0, count_f_e_0, count_e_0 = init_theta(en_train, fr_train)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


## EM

In [21]:
def train_EM(en_train, fr_train, theta_0, count_f_e_0, count_e_0, K=100):
    N = len(en_train)
    theta = theta_0
    theta_new = theta.copy()
    
    for k in range(K):
        print(k)
        count_f_e = count_f_e_0.copy()
        count_e = count_e_0.copy()
        for n in range(N):
            en_n = en_train[n]
            cleaned_en_n = preprocess(en_n)
            fr_n = fr_train[n]
            cleaned_fr_n = preprocess(fr_n)
            for i in range(len(cleaned_fr_n)):
                Z = 0
                for j in range(len(cleaned_en_n)):
                    Z += theta[cleaned_en_n[j]][cleaned_fr_n[i]]
                for j in range(len(cleaned_en_n)):
                    c = theta[cleaned_en_n[j]][cleaned_fr_n[i]] / Z
                    f_e = (cleaned_fr_n[i], cleaned_en_n[j])
                    count_f_e[f_e] += c
                    count_e[cleaned_en_n[j]] += c
        for f_e in count_f_e.keys():
            fr = f_e[0]
            en = f_e[1]
            theta_new[en][fr] = count_f_e[f_e] / count_e[en]
        
        theta = theta_new.copy()
    
    return theta_new

In [22]:
theta = train_EM(en_train, fr_train, theta_0, count_f_e_0, count_e_0, K=10)

0
1
2
3
4
5
6
7
8
9


## Test

In [10]:
theta_10.keys()

dict_keys(['th', 'parliament', 'nd', 'session', 'edited', 'hansard', 'number', 'contents', 'tuesday', 'october', 'opening', 'of', 'the', 'second', 'oaths', 'office', 'bill', 'c', 'introduction', 'and', 'first', 'reading', 'right', 'hon', 'jean', 'chr', 'é', 'tien', 'speech', 'from', 'throne', 'speaker', 'vacancies', 'york', 'west', 'mount', 'royal', 'committees', 'whole', 'appointment', 'deputy', 'chairman', 'mr', 'randy', 'white', 'supply', 'lucienne', 'robillard', 'house', 'procedure', 'affairs', 'don', 'boudria', 'address', 'in', 'reply', 'rick', 'limoges', 'ms', 'raymonde', 'folco', 'preston', 'manning', 'appendix', 'official', 'version', 'commons', 'prayers', 'which', 'had', 'been', 'prorogued', 'on', 'september', 'met', 'this', 'day', 'at', 'ottawa', 'for', 'dispatch', 'business', 'pm', 'chair', 'read', 'a', 'communication', 'secretary', 'to', 'governor', 'general', 'announcing', 'that', 'their', 'excellencies', 'john', 'ralston', 'saul', 'would', 'arrive', 'peace', 'tower', 'whe

In [18]:
a = theta_10['research']

In [19]:
sorted_x = sorted(a.items(), key=operator.itemgetter(1))

In [20]:
sorted_x[-1]

('recherche', 0.5876084536824244)